Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [72]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import time

time.sleep(0.2)


result = pd.DataFrame()

KEYWORDS = ['python', 'парсинг']

request = requests.get('https://habr.com/ru/all/')

request_parsed = BeautifulSoup(request.text, 'html.parser')

posts = soup.find_all('article', class_='post post_preview')


for post in posts:
    post_date = post.find('span', class_='post__time').text
    post_header = post.find('a', class_='post__title_link').text
    post_link = post.find('a').get('href')
    post_preview = post.find('div', class_ = 'post__text').text
    for keyword in KEYWORDS:
        if keyword in post_preview:
            row = {'Дата': post_date, 'Заголовок': post_header, 'Ссылка': post_link}
            result = pd.concat([result, pd.DataFrame([row])]) 
    
    
    
result     

""


Задание 2.

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [79]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import time

time.sleep(0.2)

email_check = "pbaholdin@mail.ru"
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {'emailAddresses': [email_check]}
headers = {'Vaar-Version': '0'}
request = requests.post(URL, params, headers)

request.text

'Missing header Vaar-Version'